In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/mihakremen/projec_secret/main/Datasets/password_dataset_full.csv?token=GHSAT0AAAAAABZ2W5SUNZGGM7BHBLRRKFBUY2I5ZFQ')
#df['Input'] = df['Input'].apply(str)
df

,Input,Target
0,Usuckballz1,1
1,Soso123aljg,1
2,Mailcreated5240,1
3,Passw0rd,1
4,8J4yE3Uz,1
...,...,...
981960,T7177306a.javan,0
981961,bytes_in,0
981962,packets_out,0
981963,nnuart1,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 981965 entries, 0 to 981964
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Input   981965 non-null  object
 1   Target  981965 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 15.0+ MB


# Определение признаков





## Вспомогательные функции

In [ ]:
import string, re

# Счетчик символов
def count(token, chartype):
  c = 0
  for char in token:
    if char in chartype:
      c += 1
  return c
 
 #Проверка на ASCII-символы
def check_ascii(text):
    for each_char in text:
        if each_char not in string.ascii_lowercase + string.ascii_uppercase + string.digits + string.punctuation:
            return False
    else:
        return True


## % кол-ва букв, цифр и символов

In [ ]:
#token = 'Aa12#!'


letter_count_up = lambda token: round(len(re.findall('[A-Z]', token)) / len(token), 2)
letter_count_low = lambda token: round(len(re.findall('[a-z]', token)) / len(token), 2)
digit_count = lambda token: round(len(re.findall("[0-9]", token)) / len(token), 2)
special_count = lambda token: round(count(token, string.punctuation) / len(token), 2)




#print(letter_count(token), digit_count(token), special_count(token)) 

## Энтропия

In [ ]:
import math

def entropy(token):
  p = 0
  Ent = 0
  for char in token:
    if char in string.ascii_uppercase:
      p = 1/26                   # англ.буквы
    elif char in string.ascii_lowercase:
      p = 1/26                   # англ.буквы
    elif char in string.digits:
      p = 1/10                       # цифры
    elif char in string.punctuation:
      p = 1/32
    Ent += p * math.log2(1/p)               
  return round(Ent, 2)

# Разбиение датасета на признаки

In [ ]:
df['Length'] = df['Input'].apply(len)
df['Uppercase'] = df['Input'].apply(letter_count_up)
df['Lowercase'] = df['Input'].apply(letter_count_low)
df['Digits'] = df['Input'].apply(digit_count)
df['Special'] = df['Input'].apply(special_count)
df['Entropy'] = df['Input'].apply(entropy)
df = df.reindex(columns=['Input', 'Length', 'Uppercase', 'Lowercase', 'Digits', 'Special', 'Entropy', 'Target'])

In [ ]:
df

,Input,Length,Uppercase,Lowercase,Digits,Special,Entropy,Target
0,Usuckballz1,11,0.09,0.82,0.09,0.00,2.14,1
1,Soso123aljg,11,0.09,0.64,0.27,0.00,2.44,1
2,Mailcreated5240,15,0.07,0.67,0.27,0.00,3.32,1
3,Passw0rd,8,0.12,0.75,0.12,0.00,1.60,1
4,8J4yE3Uz,8,0.38,0.25,0.38,0.00,1.90,1
...,...,...,...,...,...,...,...,...
981960,T7177306a.javan,15,0.07,0.40,0.47,0.07,3.75,0
981961,bytes_in,8,0.00,0.88,0.00,0.12,1.42,0
981962,packets_out,11,0.00,0.91,0.00,0.09,1.96,0
981963,nnuart1,7,0.00,0.86,0.14,0.00,1.42,0


In [ ]:
df.to_csv('/content/password_dataset.csv',index=False)

In [ ]:
# Удаление данных без символов ascii
df['not ascii'] = df['Input'].apply(check_ascii)
df = df[df['not ascii'] == True]
df.drop('not ascii', axis=1, inplace=True)
df = df.reset_index(drop=True)
df